출처: https://www.kaggle.com/code/imsewonyyy/eda-featureselection-allclassificationalgorithms/edit

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'company-bankruptcy-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1111894%2F1938459%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240930%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240930T032911Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D208340d87bf04f8e1a11cb16620195cc3d0c7935296035f4ab851c5315eca2fb006247790b6528ce411878a413be0cc82b06b24d826cfb304778d6b1afb88a6eeb5f2a606a4d7feda43cae19d6d8b050ebdded9e462ed89b86623cefcd3bf7d3932f7c0418054bef86bffafca574618045cb8bdc58270a6a31131ed18d3d9fdb02645e7316f8bf52496162178b03673a0db2fbb79ffc02362beb24ebf527fc01dfd95be0c6572abd05617821e602a7b6f614b4bb664eea73820af948dce320ac4152fd733448d47150976cfd717a2a8b8b76a93b343668d166e4c0cb355bfde61690550a9230b0af2aaa6a27b0eee0800764275b0b3d4b4c4229a4fc01cadf53'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


속성 정보:
총 95개의 기능이 있습니다. 이 데이터 세트에는 많은 기능이 있습니다. 데이터 세트 설명은 데이터 페이지 자체에 제공됩니다. EDA를 맹목적으로 수행하기 전에 데이터에 대한 정보를 확보하는 것이 중요합니다.

https://www.kaggle.com/fedesoriano/company-bankruptcy-prediction

먼저 원시 데이터에 대해 모델을 훈련하고, 특징 선택 기법을 사용하여 일부 기능을 강조 표시하고 선택한 기능에 대해 훈련할 것입니다. 따라서 모델과 정확도를 비교할 것입니다.

우리 플랜
1. 데이터 세트 관찰
2. 탐색적 데이터 분석

2.1 데이터 세트 청소
2.2 데이터 불균형 확인
3. 데이터 전처리

3.2 교육 및 테스트 분할
3.2 랜덤 포레스트 기능 선택
3.3 PCA
4. 모델, 하이퍼파라미터 튜닝, 교차 검증 및 모델 평가

4.1 로지스틱 회귀 분석
4.2 나이브 베이즈
4.3 K-가장 가까운 이웃
4.4 의사 결정 트리
4.5 랜덤 포레스트
4.6 XGBoost


-
    


# 1. 데이터세트 관찰

In [ ]:
import pandas as pd
pd.set_option('max_columns', None)

In [ ]:
path = "../input/company-bankruptcy-prediction/"

In [ ]:
df = pd.read_csv(path + 'data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
corr = df.corr()

In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
sns.heatmap(corr, ax = ax, cmap = 'viridis', linewidth = 0.1)

In [ ]:
df.info()

관찰
모든 기능은 숫자입니다(int64 또는 float64)
모든 값은 -1에서 1 사이로 조정됩니다.

탐색적 데이터 분석

2.1 데이터 불균형 확인

2.2 이상치

2.3 null 값 채우기


### 2.1 데이터 불균형 확인

In [ ]:
df['Bankrupt?'].value_counts()

In [ ]:
print('Financially stable:', round(df['Bankrupt?'].value_counts()[0] / len(df) * 100,2) ,'%')
print('Financially unstable:', round(df['Bankrupt?'].value_counts()[1] / len(df) * 100, 2), '%')

우리는 데이터가 재정적으로 안정적이라는 방향으로 매우 왜곡되어 있음을 알 수 있습니다. 이 데이터 세트에서 모델을 훈련하면 예측이 재정적으로 안정적이라는 방향으로 편향될 것입니다.

우리는 모델을 훈련하기 위해 데이터 세트의 균형을 맞출 것입니다.

참고: 원래 데이터 세트가 얼마나 불균형한지 알아보세요! 대부분의 회사는 재무적으로 안정적입니다. 이 데이터 프레임을 예측 모델과 분석의 기반으로 사용하면 많은 오류가 발생할 수 있으며, 대부분의 회사가 재무적으로 안정적이라고 "추정"하기 때문에 알고리즘이 과적합될 가능성이 높습니다. 하지만 우리는 모델이 파산 징후를 보이는 패턴을 감지하지 않기를 바랍니다!

In [ ]:
## Visualizing the datas

sns.set_theme(context = 'paper')


plt.figure(figsize = (8,8))
sns.countplot(x = 'Bankrupt?', data = df);
plt.title('Class Distributions: \n 0: Financially Stable & 1: Financially Unstable');

데이터 분할(오리지널 데이터프레임)
랜덤언더샘플링 기법을 진행하기 전에 원본 데이터 프레임을 분리해야 합니다.

왜요?

테스트 목적으로 무작위 언더샘플링 또는 오버샘플링 기술을 구현할 때 데이터를 분할하고 있지만, 이러한 기술 중 어느 것도 생성한 테스트 세트가 아닌 원래 테스트 세트에서 모델을 테스트하고자 합니다. 주요 목표는 언더샘플링 및 오버샘플링된 데이터 프레임에 모델을 맞추고 원래 테스트 세트에서 테스트하는 것입니다(모델이 패턴을 감지할 수 있도록).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold

print("Financially Stable:", round(df['Bankrupt?'].value_counts()[0] / len(df) * 100, 2), '% of the dataset')
print("Financially Unstable:", round(df['Bankrupt?'].value_counts()[1] / len(df) * 100,2),'% of the dataset')

X = df.drop('Bankrupt?', axis = 1)
y = df['Bankrupt?']

sss = StratifiedKFold(n_splits = 5, random_state = None, shuffle = False)

for train_index, test_index in sss.split(X,y):
    print("\n Train", train_index, "Test", test_index)
    org_Xtrain, org_Xtest = X.iloc[train_index], X.iloc[test_index]
    org_ytrain, org_ytest = y.iloc[train_index], y.iloc[test_index]


In [ ]:
import numpy as np

In [ ]:
## turn into an array

org_Xtrain = org_Xtrain.values
org_Xtest = org_Xtest.values
org_ytrain = org_ytrain.values
org_ytest = org_ytest.values

## See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(org_ytrain, return_counts = True)
test_unique_label, test_counts_label = np.unique(org_ytest, return_counts = True)

print('Label Distirubtions: \n')
print(train_counts_label / len(org_ytrain))
print(test_counts_label / len(org_ytest))

무작위 언더샘플링 및 오버샘플링

프로젝트의 이 단계에서는 보다 균형 잡힌 데이터 세트를 확보하기 위해 기본적으로 데이터를 제거하고 모델이 과적합되지 않도록 하는 '랜덤 언더 샘플링'을 구현할 것입니다.


In [ ]:
## Lets shuffle the data before creating the subsamples

xdf = df.sample(frac = 1)

## amount of Financially unstable data is 220
# sdf = Financially stable
# ndf = Financially unstable

sdf = df.loc[xdf['Bankrupt?'] == 0][:220]
ndf = df.loc[xdf['Bankrupt?']==1]

normal_distributed_df = pd.concat([sdf, ndf])

# Shuffling again

nxdf = normal_distributed_df.sample(frac = 1, random_state = 42)

In [ ]:
nxdf.head()

In [ ]:
## Checking new dataframe

print("Distribution of the Classes in the subsample dataset")
print(nxdf['Bankrupt?'].value_counts() / len(nxdf))

sns.countplot('Bankrupt?', data = nxdf)
plt.title("Equally Distributed Class", fontsize = 14)
plt.show()

#### 상관 행렬

상관 행렬은 데이터를 이해하는 데 있어 핵심입니다. 특정 거래가 사기인지 여부에 큰 영향을 미치는 기능이 있는지 알고 싶습니다. 그러나 사기 거래와 관련하여 어떤 기능이 높은 양의 상관관계를 갖는지 확인하려면 올바른 데이터 프레임(하위 샘플)을 사용하는 것이 중요합니다.

In [ ]:
## make sure we use the subsampe in our correlation

f, (ax1, ax2) = plt.subplots(2,1, figsize = (54,50))

## Entire data frame

corr = df.corr()
sns.heatmap(corr, cmap = 'coolwarm_r', annot_kws = {'size': 20}, ax= ax1)
ax1.set_title("Imbalanced Correlated Matrix \n")


sub_sample_corr = nxdf.corr()
sns.heatmap(sub_sample_corr, cmap = 'coolwarm_r', annot_kws = {'size': 20}, ax = ax2)
ax2.set_title("SubSample Correlation Matrix")
plt.show()

In [ ]:
nxdf.hist(bins = 50, figsize = (35,20))
plt.show()


파란색 사각형 상자와 빨간색 사각형 상자가 많이 있는데, 이는 해당 열이 하나 또는 다른 열과 상관관계가 높거나 낮음을 나타냅니다. 따라서 PCA(차원 감소) 기술 큐를 사용합니다.

PCA 대 기능 선택?

https://stackoverflow.com/questions/16249625/difference-between-pca-principal-component-analysis-and-feature-selection#:~:text=The%20difference%20is%20that%20PCA,takes%20the%20target%20into%20consideration.&text=PCA%20is%20based%20on%20extracting,data%20shows%20the%20highest%20variability.

위의 매우 좋은 답변을 덧붙이자면 다음과 같습니다. 차이점은 PCA가 데이터의 한 특징이 다른 특징(선형 종속성)으로 어떻게 표현되는지 탐구하여 차원을 줄이려고 한다는 것입니다. 대신 특징 선택은 대상을 고려합니다. 입력 변수의 순위를 목표 값을 예측하는 데 얼마나 유용한지에 따라 결정됩니다. 이는 일변량 특징 선택의 경우에도 마찬가지입니다. 다중 변수 특징 선택은 입력의 일부 특징을 폐기한다는 의미에서 PCA의 한 형태로 간주할 수 있는 작업도 수행할 수 있습니다. 하지만 이 비유를 너무 멀리 받아들이지 마세요.

## 3. 데이터 전처리

- 교육 및 테스트 분할
- 랜덤 포레스트를 사용한 기능 선택
- PCA

#### 교육 및 테스트 분할


In [ ]:
## this is equally sampled dataset (perfectly balanced target)

X = nxdf.drop(['Bankrupt?'],1)
y = nxdf['Bankrupt?']

rf_fs_Xtrain, rf_fs_Xtest, rf_fs_ytrain, rf_fs_ytest = train_test_split(X,y, test_size = 0.1, random_state = 1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

### 랜덤 포레스트를 사용한 기능 선택

In [ ]:
## modelling with balanced traget

model = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
model.fit(rf_fs_Xtrain, rf_fs_ytrain)

sel = SelectFromModel(model)

In [ ]:
## balanced target

sel.fit(rf_fs_Xtrain, rf_fs_ytrain)

In [ ]:
# balanced

selected_feat= rf_fs_Xtrain.columns[(sel.get_support())]
len(selected_feat)

In [ ]:
selected_feat

In [ ]:
## Creating a dataframe for only selected values to train later

rf_fs = pd.DataFrame()

for column in selected_feat:
    if column in nxdf:
        rf_fs[column] = nxdf[column].values



In [ ]:
rf_fs

### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_components = 2
pca = PCA(n_components = n_components)
pca.fit(nxdf)

In [ ]:
X_reduced_pca = PCA(n_components=2, random_state=42).fit_transform(rf_fs_Xtrain.values)

In [ ]:
x_pca = pca.transform(nxdf)

In [ ]:
x_pca.shape

In [ ]:
# PCA scatter plot
plt.figure(figsize = (8,8))
plt.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=(rf_fs_ytrain == 0), cmap='coolwarm', label= 'Stable_Company', linewidths=2)
plt.scatter(X_reduced_pca[:,0], X_reduced_pca[:,1], c=(rf_fs_ytrain == 1), cmap='coolwarm', label= 'Unstable_Company', linewidths=2)
plt.show()

## 모델 테스트

우리는 모든 데이터 세트(즉, 각 모델에 대한 일반적인 랜덤 포레스트 특징 선택 및 PCA 데이터 세트)를 테스트할 것입니다.

비교를 위해 새로운 데이터프레임을 만들고 어떤 방법이 더 나은 성능을 보였는지 비교할 것입니다

또한 분류 문제이므로 다음과 같은 알고리즘으로 테스트합니다

- 로지스틱 회귀 분석
- 나이브 베이즈
- KNN
- 의사 결정 트리
- 랜덤 포레스트
- SVM

#### 모델에 대한 모든 데이터 세트 준비하기

- <b> nxdf </b>는 원래 데이터 세트입니다.
- <b> rf_fs </b>는 랜덤 포레스트에서 피처 선택이 가능한 데이터 세트입니다


In [ ]:
## Splitting dataset for Normal data without feature selection

X_train, X_test, y_train, y_test = train_test_split(nxdf.drop('Bankrupt?', axis = 1), nxdf['Bankrupt?'],test_size = 0.1, random_state = 1)

In [ ]:
nxdf.shape

In [ ]:
rf_fs.shape

- <b> rf_fs </b> 목표 기능 <b> 파산? </b>는 이미 삭제되었습니다. nxdf와 rf_fs의 목표 값이 동일하다는 것을 알고 있으므로 선택한 데이터 세트를 분할하는 데 nxdf의 목표 값을 사용합니다

In [ ]:
## Splitting RandomForest Feature Selection dataset

fs_Xtrain, fs_Xtest, fs_ytrain, fs_ytest = train_test_split(rf_fs, nxdf['Bankrupt?'], test_size = 0.1, random_state = 1)

In [ ]:
model_score = pd.DataFrame(columns = ("Original_Dataset","Selected_Dataset"))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, confusion_matrix

### Original Dataset

In [ ]:
lrmodel1 = LogisticRegression(max_iter = 1000)
lrmodel1.fit(X_train, y_train)
score1 = lrmodel1.score(X_test, y_test)
lr_pred1 = lrmodel1.predict(X_test)

In [ ]:
## Accuracy on Original Datset without Feature Selection:

print("Score:", score1)

In [ ]:
lr_cm1 = confusion_matrix(y_test, lr_pred1, labels = (1,0))

In [ ]:
lr_cm1

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(lr_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
y_test.value_counts()

### 기능 선택 데이터 세트

In [ ]:
lrmodel2 = LogisticRegression(max_iter = 1000)
lrmodel2.fit(fs_Xtrain, fs_ytrain)
score2 = lrmodel2.score(fs_Xtest, fs_ytest)
lr_ypred2 = lrmodel2.predict(fs_Xtest)

In [ ]:
print("Score", score2)

In [ ]:
lr_cm2 = confusion_matrix(fs_ytest, lr_ypred2, labels = (1,0))
print("Confusion Matrix: \n", lr_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(lr_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
model_score = model_score.append(pd.DataFrame({'Original_Dataset':[score1], 'Selected_Dataset': [score2]}, index = ['LogisticRegression']))

In [ ]:
model_score

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

### Original dataset

In [ ]:
naiveb1 = GaussianNB()

In [ ]:
naiveb1.fit(X_train, y_train)
score1 = naiveb1.score(X_test, y_test)
nb_pred1 = naiveb1.predict(X_test)

In [ ]:
print("Score:", score1)

In [ ]:
nb_cm1 = confusion_matrix(y_test, nb_pred1, labels = (1,0))

In [ ]:
print("Confusion Matrix: \n", nb_cm1)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(nb_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

### Feature Selection Dataset

In [ ]:
naiveb2 = GaussianNB()

In [ ]:
naiveb2.fit(fs_Xtrain, fs_ytrain)
score2 = naiveb2.score(fs_Xtest, fs_ytest)
nb_pred2 = naiveb2.predict(fs_Xtest)

In [ ]:
print("Score:", score2)

In [ ]:
nb_cm2 = confusion_matrix(fs_ytest, nb_pred2, labels = [1,0])
print("Confusion Matrix: \n", nb_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(nb_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
model_score = model_score.append(pd.DataFrame({'Original_Dataset': [score1], 'Selected_Dataset': [score2]}, index = ['NaiveBayes']))

In [ ]:
model_score

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### Original Dataset

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors = 7)

In [ ]:
knn1.fit(X_train, y_train)

In [ ]:
score1 = knn1.score(X_test, y_test)
print(score1)

In [ ]:
knn_pred1 = knn1.predict(X_test)
knn_cm1 = confusion_matrix(y_test, knn_pred1, labels = (1,0))
print("Confusion Matrix:\n", knn_cm1)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(knn_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
### Hyperparameter tuning for KNN

In [ ]:
error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train, y_train)
    pred_knn = knn.predict(X_test)
    error_rate.append(np.mean(pred_knn != y_test))

In [ ]:
plt.figure(figsize = (8,8))
plt.plot(range(1,40), error_rate, color = 'blue', linestyle = 'dashed', marker = 'o', markerfacecolor = 'red', markersize = 10);
plt.title('Error Rate vs K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
## let's see how much difference does it makes

tuned_knn1 = KNeighborsClassifier(n_neighbors = 4)
tuned_knn1.fit(X_train, y_train)

In [ ]:
tuned_score1 = tuned_knn1.score(X_test, y_test)
print(tuned_score1)

We can see, it's not that different

### Feature Selection Dataset

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors = 7)
knn2.fit(fs_Xtrain, fs_ytrain)

In [ ]:
score2 = knn2.score(fs_Xtest, fs_ytest)
print(score2)

In [ ]:
knn_pred2 = knn2.predict(fs_Xtest)
knn_cm2 = confusion_matrix(fs_ytest, knn_pred2, labels = (1,0))
print("Confusion Matrix: \n", knn_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(knn_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
## Hyperparamter tuning for this

In [ ]:
error_rate = []

for i in range(1,40):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(fs_Xtrain, fs_ytrain)
    pred_knn = knn.predict(fs_Xtest)
    error_rate.append(np.mean(pred_knn != fs_ytest))

In [ ]:
plt.figure(figsize = (8,8))
plt.plot(range(1,40), error_rate, color = 'blue', linestyle = 'dashed', marker = 'o', markerfacecolor = 'red', markersize = 10);
plt.title('Error Rate vs K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

K = 5로 테스트해 보겠습니다

In [ ]:
tuned_knn2 = KNeighborsClassifier(n_neighbors = 14)
tuned_knn2.fit(fs_Xtrain, fs_ytrain)

In [ ]:
tuned_score2 = tuned_knn2.score(fs_Xtest, fs_ytest)
print(tuned_score2)

In [ ]:
tuned_knn_pred2 = knn.predict(fs_Xtest)
tuned_cm2 = confusion_matrix(fs_ytest, tuned_knn_pred2, labels = (1,0))
print("Confusion Matrix: \n", tuned_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(tuned_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

A bit better but not that great

In [ ]:
model_score = model_score.append(pd.DataFrame({'Original_Dataset': [tuned_score1], 'Selected_Dataset': [tuned_score2]}, index = ['KNN']))


In [ ]:
model_score

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt1 = DecisionTreeClassifier()

### Original data

In [ ]:
dt1 = dt1.fit(X_train, y_train)

In [ ]:
score1 = dt1.score(X_test, y_test)
print(score1)

In [ ]:
dt_pred1 = dt1.predict(X_test)
dt_cm1 = confusion_matrix(y_test, dt_pred1, labels = [1,0])
print("Confusion Matrix: \n", dt_cm1)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(dt_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

### Feature Selection Data

In [ ]:
dt2 = DecisionTreeClassifier()

In [ ]:
dt2 = dt2.fit(fs_Xtrain, fs_ytrain)

In [ ]:
score2 = dt2.score(fs_Xtest, fs_ytest)
print(score2)

In [ ]:
dt_pred2 = dt2.predict(fs_Xtest)
dt_cm2 = confusion_matrix(fs_ytest, dt_pred2, labels = [1,0])
print("Confusion Matrix: \n", dt_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(dt_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
model_score = model_score.append(pd.DataFrame({'Original_Dataset': [score1], 'Selected_Dataset': [score2]}, index = ['DecisionTrees']))


In [ ]:
model_score

# Random Forest

In [ ]:
rfclf1 = RandomForestClassifier(n_estimators = 100)

### With Original dataset

In [ ]:
rfclf1.fit(X_train, y_train)

In [ ]:
score1 = rfclf1.score(X_test, y_test)
print(score1)

In [ ]:
rf_pred1 = rfclf1.predict(X_test)
rf_cm1 = confusion_matrix(y_test, rf_pred1, labels = (1,0))
print("Confusion Matrix: \n", rf_cm1)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(rf_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

### With Selected Features

In [ ]:
rfclf2 = RandomForestClassifier(n_estimators = 100)

In [ ]:
rfclf2.fit(fs_Xtrain, fs_ytrain)

In [ ]:
score2 = rfclf2.score(fs_Xtest, fs_ytest)
print(score2)

In [ ]:
rf_pred2 = rfclf2.predict(fs_Xtest)
rf_cm2 = confusion_matrix(fs_ytest, rf_pred2, labels = [1,0])
print("Confusion Matrix: \n", rf_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(rf_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

#### Hyperparamter Tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)
               ]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [2,4]

# Minimum number of samples required to split a node
min_samples_split = [2, 5]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

In [ ]:
tuned_rf = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = tuned_rf, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [ ]:
rf_Grid.fit(X_train, y_train)

In [ ]:
rf_Grid.best_params_

In [ ]:
print (f'Train Accuracy - : {rf_Grid.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {rf_Grid.score(X_test,y_test):.3f}')

In [ ]:
tuned_score2 = rf_Grid.score(X_test, y_test)
print(tuned_score2)

In [ ]:
model_score = model_score.append(pd.DataFrame({'Original_Dataset': [score1], 'Selected_Dataset': [tuned_score2]}, index = ['RandomForest']))

In [ ]:
model_score

# XGBoost

In [ ]:
from xgboost import XGBClassifier

### With Original Dataset

In [ ]:
xgb1 = XGBClassifier(n_estimators = 100)
xgb1.fit(X_train, y_train)

In [ ]:
score1 = xgb1.score(X_test, y_test)
print(score1)

In [ ]:
xgb_pred1 = xgb1.predict(X_test)
xgb_cm1 = confusion_matrix(y_test, xgb_pred1, labels = [1,0])
print("Confusion Matrix: \n", xgb_cm1)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(xgb_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

### With Selected Dataset

In [ ]:
xgb2 = XGBClassifier(n_estimators = 100)
xgb2.fit(fs_Xtrain, fs_ytrain)

In [ ]:
score2 = xgb2.score(fs_Xtest, fs_ytest)
print(score2)

In [ ]:
xgb_pred2 = xgb2.predict(fs_Xtest)
xgb_cm2 = confusion_matrix(fs_ytest, xgb_pred2, labels = [1,0])
print("Confusion Matrix: \n", xgb_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(xgb_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

### Hyperparamter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]

}

In [ ]:
tuned_xgb = XGBClassifier()

In [ ]:
random_search = RandomizedSearchCV(tuned_xgb, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = 1, cv = 5, verbose = 3)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:
tuned_score1 = random_search.score(X_test, y_test)
print(tuned_score1)

In [ ]:
print (f'Train Accuracy - : {random_search.score(X_train,y_train):.3f}')
print (f'Test Accuracy - : {random_search.score(X_test,y_test):.3f}')

In [ ]:
tuned_xgb_pred1 = random_search.predict(X_test)
tuned_xgb_cm1 = confusion_matrix(y_test, tuned_xgb_pred1, labels = [1,0])
print("Confusion Matrix: \n", tuned_xgb_cm1)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(tuned_xgb_cm1, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

확실히 이것은 포스트 클래스에 편향되어 있습니다. 이것은 불균형한 데이터 세트이기 때문입니다. 우리는 동등하게 균형 잡힌 데이터 세트로 하이프루닝할 것입니다

### Hypertuning for balanced dataset

In [ ]:
tuned_xgb2 = XGBClassifier()

In [ ]:
random_search2 = RandomizedSearchCV(tuned_xgb2, param_distributions = params, n_iter = 5, scoring = 'roc_auc', n_jobs = 1, cv = 5, verbose = 3)

In [ ]:
random_search2.fit(fs_Xtrain, fs_ytrain)

In [ ]:
random_search2.best_estimator_

In [ ]:
random_search2.best_params_

In [ ]:
tuned_score2 = random_search2.score(fs_Xtest, fs_ytest)
print(tuned_score2)

In [ ]:
print (f'Train Accuracy - : {random_search2.score(fs_Xtrain,fs_ytrain):.3f}')
print (f'Test Accuracy - : {random_search2.score(fs_Xtest,fs_ytest):.3f}')

In [ ]:
tuned_xgb_pred2 = random_search2.predict(fs_Xtest)
tuned_xgb_cm2 = confusion_matrix(fs_ytest, tuned_xgb_pred2, labels = [1,0])
print("Confusion Matrix: \n", tuned_xgb_cm2)

In [ ]:
x_axis_labels = [1,0]
y_axis_labels = [1,0]

sns.set(font_scale=1.4)
sns.heatmap(tuned_xgb_cm2, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot = True, annot_kws = {'size': 16})
plt.xlabel("Actual Class")
plt.ylabel("Predicted Class")
plt.show()

In [ ]:
model_score = model_score.append(pd.DataFrame({'Original_Dataset': [tuned_score1], 'Selected_Dataset': [tuned_score2]}, index = ['XGBoost']))


In [ ]:
model_score

보시다시피 <b> XGBoost </b>는 선택한 기능 </b>에서 가장 우수한 성능을 발휘합니다.

In [ ]:
## Checking Classification report of the best model

print(classification_report(fs_ytest, tuned_xgb_pred2))

In [ ]:
## Biased model

print(classification_report(y_test, tuned_xgb_pred1))

In [ ]:
### Checking Classification report of the worst model

print(classification_report(y_test, lr_ypred2 ))